In [1]:
class State(object):
    """ Boat=1 means boat is on the left side and Boat=0 means boat is on the right side"""
    def __init__(self, missionaries, cannibals, boat, by_move):
        self.missionaries = missionaries
        self.cannibals = cannibals
        self.boat = boat
        self.by_move = by_move
        
    def __str__(self):
        return "%s, %s %s %s" %(self.by_move, self.missionaries, self.cannibals, self.boat)
        
    def __eq__(self, other):
        return self.__str__() == other.__str__()
    
    def __hash__(self):
        return hash(self.__str__())

    def is_valid(self):
        if self.missionaries < 0 or self.missionaries > 3:
            return False
        if self.cannibals < 0 or self.cannibals > 3:
            return False
        if self.boat > 1 or self.boat < 0:
            return False
        if self.missionaries < self.cannibals and self.missionaries > 0:
            return False
        if self.missionaries > self.cannibals and self.missionaries < 3:
            return False

        return True
        
    def is_goal(self):
        return self.missionaries == 0 and self.cannibals == 0 and self.boat == 0

    def new_states(self):
        op = -1 # Subtract
        boat_move = "from left bank to right"
        if self.boat == 0:
            op = 1 # Add
            boat_move = "from right bank to left"

        for x in range(3):
            for y in range(3):
                by_move = "Move %s missionaries and %s cannibals %s" %(x, y, boat_move)
                new_state = State(self.missionaries + op*x, self.cannibals + op*y, self.boat + op*1, by_move)
                if x+y >= 1 and x+y <= 2 and new_state.is_valid():
                    yield new_state

In [2]:
class Node(object):
    def __init__(self, parent, state, depth):
        self.parent = parent
        self.state = state
        self.depth = depth

    def __str__(self):
        return self.state.__str__()
    
    def __eq__(self, other):
        return self.__str__() == other.__str__()
    
    def __hash__(self):
        return hash(self.__str__())

    def childrens(self):
        for state in self.state.new_states():
            yield Node(parent=self, state=state, depth=self.depth+1)

    def extract_solution(self):
        solution = []
        node = self
        solution.append(node)
        while node.parent is not None:
            solution.append(node.parent)
            node = node.parent
        solution.reverse()
        return solution

In [3]:
def dfs(node):
    duplicates = []
    stack = [ node ]
    while len(stack) != 0:
        current = stack.pop()
        if str(current) in duplicates:
            continue
        duplicates.append(str(current))
        
        if current.state.is_goal():
            return current.extract_solution()
        for child in current.childrens(): 
            stack.append(child) 

In [4]:
initial_state = State(3, 3, 1, "Initial State")
root = Node(parent=None, state=initial_state, depth=0)
for state in dfs(root):
    print(state)

Initial State, 3 3 1
Move 1 missionaries and 1 cannibals from left bank to right, 2 2 0
Move 1 missionaries and 1 cannibals from right bank to left, 3 3 1
Move 0 missionaries and 2 cannibals from left bank to right, 3 1 0
Move 0 missionaries and 1 cannibals from right bank to left, 3 2 1
Move 1 missionaries and 0 cannibals from left bank to right, 2 2 0
Move 1 missionaries and 0 cannibals from right bank to left, 3 2 1
Move 0 missionaries and 2 cannibals from left bank to right, 3 0 0
Move 0 missionaries and 1 cannibals from right bank to left, 3 1 1
Move 2 missionaries and 0 cannibals from left bank to right, 1 1 0
Move 1 missionaries and 1 cannibals from right bank to left, 2 2 1
Move 2 missionaries and 0 cannibals from left bank to right, 0 2 0
Move 0 missionaries and 1 cannibals from right bank to left, 0 3 1
Move 0 missionaries and 2 cannibals from left bank to right, 0 1 0
Move 1 missionaries and 0 cannibals from right bank to left, 1 1 1
Move 1 missionaries and 1 cannibals from 